In [1]:
import pandas as pd
df = pd.read_csv('data/france_2017.csv')
df
winner_names = []
loser_names = []
for winner_name, loser_name in zip(df[df.columns[4]].str.split(), df[df.columns[8]].str.split()):
#     print("UWU")
    winner_names.append(' '.join(winner_name[1:]))
    loser_names.append(' '.join(loser_name[1:]))
    
df[df.columns[4]] = winner_names
df[df.columns[8]] = loser_names    
df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10
0,0,France,2017,XX,Xavier BRETON,LR,"53,75",True,Laurent MALLET,MDM,"46,25",False
1,1,France,2017,XX,Charles DE LA VERPILLIÈRE,LR,"51,72",True,Marie Jeanne BEGUET,MDM,"48,28",False
2,2,France,2017,XX,Olga GIVERNET,REM,"61,86",True,Stéphanie PERNOD BEAUDON,LR,"38,14",False
3,3,France,2017,XX,Stéphane TROMPILLE,REM,"64,50",True,Blanche CHAUSSAT,FN,"35,50",False
4,4,France,2017,XX,Damien ABAD,LR,"67,02",True,Hélène DE MEIRE,REM,"32,98",False


In [7]:
winners = df[[df.columns[4], df.columns[5]]]
losers = df[[df.columns[8], df.columns[9]]]
losers.columns = ['candidate', 'party']
winners.columns = ['candidate', 'party']
all_candidates = pd.concat([winners, losers])
all_candidates
all_candidates['candidate'] = all_candidates['candidate'].astype(str)
all_candidates['party'] = all_candidates['party'].astype(str)
all_candidates = all_candidates.applymap(lambda x: x.strip() if isinstance(x, str) else x)
all_candidates.drop_duplicates(inplace = True, subset = ['candidate', 'party'])
all_candidates                                    

,candidate,party
0,"Dr. Kaufmann, Stefan",CDU
1,"Maag, Karin",CDU
2,"Biadacz, Marc",CDU
3,"Grübel, Markus",CDU
4,"Hennrich, Michael",CDU
5,"Färber, Hermann",CDU
6,"Dr. Pfeiffer, Joachim",CDU
7,"Bilger, Steffen",CDU
8,"Gienger, Eberhard",CDU
9,"Throm, Alexander",CDU


In [4]:
all_candidates['candidate'] = all_candidates['candidate']+' candidat'
search_terms = all_candidates['candidate']


In [4]:
from googleapiclient.discovery import build
import pprint
import pandas as pd
from PIL import Image
import requests
from io import BytesIO
import cv2
import numpy as np
import os
import time
import requests
from PIL import Image
from io import BytesIO
from keys_file import GOOGLE_KEY
def get_imgs_from_urls(img_urls):
    images = []
    for image_url in img_urls:
        try:
            response = requests.get(image_url)
            image = Image.open(BytesIO(response.content))
            images.append(image)
        except: 
            None
    return images

def get_image_from_json(json):
    images = []
    for item in json['value']:
        try:
            response = requests.get(item['contentUrl'])
            image = Image.open(BytesIO(response.content))
            images.append(image)
        except: 
            None
    return images
def count_faces(image):
    xml_files = ['xml_files/haarcascade_frontalface_default.xml','xml_files/lbpcascade_frontalface.xml']
    faces_counted = []
    for xml_file in xml_files:
        face_cascade = cv2.CascadeClassifier(xml_file)
        gray = cv2.cvtColor(np.float32(image), cv2.COLOR_BGR2GRAY)
        gray = np.array(gray, dtype='uint8')
        faces = face_cascade.detectMultiScale(gray, 1.2, 5)
        faces_counted.append(len(faces))
        if len(faces) == 1:
            return 1
            break
    return 0
        
# MTCNN uses depricted functions.
def count_faces_dnn(image):
    img = cv2.cvtColor(np.float32(image), cv2.COLOR_BGR2RGB)
    detector = MTCNN()
    faces = detector.detect_faces(img)
    print(len(faces))    
    return None
        

def check_directory(query):
    directory = 'images france 2017/'+query
    if not os.path.exists(directory):
        os.makedirs(directory)
    return directory

def select_relevant_images(images):
#   check if url contains portrait
    relevant_images = []
    for image  in images:
        faces_count = 0
        try:
            faces_count = count_faces(image)
        except:
            None
        if faces_count == 1:
            relevant_images.append(image)
    return relevant_images

def write_images_to_directory(directory, images, query):
    counter = 1
    for image in images:
        if not image.mode == 'RGB':
            image = image.convert('RGB')
        image.save(directory+"/google_"+query+str(counter)+".jpeg")
        counter += 1;
        
def get_google_urls(search_term):
    service = build("customsearch", "v1",
            developerKey= GOOGLE_KEY)

    res = service.cse().list(
      q=search_term,
      cx='014171572285050841134:z8jpb8kffjb',
      num='8',
      searchType='image',
      imgType='face'
    ).execute()
    urls = []
    for item in res['items']:
#         print(item['link'])
        urls.append(item['link'])
    return urls


In [5]:
counter = 0
for search_term in search_terms:
    print(search_term)
    print(counter, "of", len(search_terms))
    urls = get_google_urls(search_term)
    if urls:
        images = get_imgs_from_urls(urls)
        directory = check_directory(search_term)
#         images = select_relevant_images(images)
        write_images_to_directory(directory, images, search_term)
    else:
        print('No images found') 
    time.sleep(1)
    counter += 1
print("DONE")

Xavier BRETON candidat
0 of 1114
Charles DE LA VERPILLIÈRE candidat
1 of 1114
Olga GIVERNET candidat
2 of 1114
Stéphane TROMPILLE candidat
3 of 1114
Damien ABAD candidat
4 of 1114
Aude BONO-VANDORME candidat
5 of 1114
Julien DIVE candidat
6 of 1114
Jean-Louis BRICOUT candidat
7 of 1114
Marc DELATTE candidat
8 of 1114
Jacques KRABAL candidat
9 of 1114
Jean-Paul DUFRÈGNE candidat
10 of 1114
Laurence VANCEUNEBROCK-MIALON candidat
11 of 1114
Bénédicte PEYROL candidat
12 of 1114
Delphine BAGARRY candidat
13 of 1114
Christophe CASTANER candidat
14 of 1114
Pascale BOYER candidat
15 of 1114
Joël GIRAUD candidat
16 of 1114
Eric CIOTTI candidat
17 of 1114
Loîc DOMBREVAL candidat
18 of 1114
Cédric ROUSSEL candidat
19 of 1114
Alexandra VALETTA ARDISSON candidat
20 of 1114
Marine BRENIER candidat
21 of 1114
Laurence TRASTOUR-ISNART candidat
22 of 1114
Eric PAUGET candidat
23 of 1114
Bernard BROCHAND candidat
24 of 1114
Michèle TABAROT candidat
25 of 1114
Hervé SAULIGNAC candidat
26 of 1114
Olivier 

/Users/olafkroon/anaconda3/lib/python3.7/site-packages/PIL/Image.py:2821: UserWarning: image file could not be identified because WEBP support not installed
  warnings.warn(message)


Anne-France BRUNET candidat
206 of 1114
Aude AMADOU candidat
207 of 1114
Sarah EL HAÏRY candidat
208 of 1114
Yves DANIEL candidat
209 of 1114
Sandrine JOSSO candidat
210 of 1114
Audrey DUFEU SCHUBERT candidat
211 of 1114
Yannick HAURY candidat
212 of 1114
Sophie ERRANTE candidat
213 of 1114
Stéphanie RIST candidat
214 of 1114
Caroline JANVIER candidat
215 of 1114
Claude DE GANAY candidat
216 of 1114
Jean-Pierre DOOR candidat
217 of 1114
Marianne DUBOIS candidat
218 of 1114


/Users/olafkroon/anaconda3/lib/python3.7/site-packages/PIL/Image.py:993: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Richard RAMOS candidat
219 of 1114
Aurélien PRADIÉ candidat
220 of 1114
Huguette TIEGNA candidat
221 of 1114
Michel LAUZZANA candidat
222 of 1114
Alexandre FRESCHI candidat
223 of 1114
Olivier DAMAISIN candidat
224 of 1114
Pierre MOREL A L'HUISSIER candidat
225 of 1114
Matthieu ORPHELIN candidat
226 of 1114
Stella DUPONT candidat
227 of 1114
Jean-Charles TAUGOURDEAU candidat
228 of 1114
Laëtitia SAINT-PAUL candidat
229 of 1114
Denis MASSEGLIA candidat
230 of 1114
Nicole DUBRE-CHIRAT candidat
231 of 1114
Philippe BOLO candidat
232 of 1114
Philippe GOSSELIN candidat
233 of 1114
Bertrand SORRE candidat
234 of 1114
Stéphane TRAVERT candidat
235 of 1114
Sonia KRIMI candidat
236 of 1114
Valérie BEAUVAIS candidat
237 of 1114
Aina KURIC candidat
238 of 1114
Éric GIRARDIN candidat
239 of 1114
Lise MAGNIER candidat
240 of 1114
Charles DE COURSON candidat
241 of 1114
Bérangère ABBA candidat
242 of 1114
François CORNUT-GENTILLE candidat
243 of 1114
Guillaume GAROT candidat
244 of 1114
Géraldine BA

...

/Users/olafkroon/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:804: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


Patricia GALLERNEAU candidat
443 of 1114
Stéphane BUCHOU candidat
444 of 1114
Martine LEGUILLE-BALLOY candidat
445 of 1114
Pierre HENRIET candidat
446 of 1114
Jacques SAVATIER candidat
447 of 1114
Sacha HOULIÉ candidat
448 of 1114
Jean-Michel CLÉMENT candidat
449 of 1114
Nicolas TURQUOIS candidat
450 of 1114
Sophie BEAUDOUIN-HUBIERE candidat
451 of 1114
Jean-Baptiste DJEBBARI-BONNET candidat
452 of 1114
Marie-Ange MAGNE candidat
453 of 1114
Stéphane VIRY candidat
454 of 1114
Gérard CHERPION candidat
455 of 1114
Christophe NAEGELEN candidat
456 of 1114
Jean-Jacques GAULTIER candidat
457 of 1114
Guillaume LARRIVÉ candidat
458 of 1114
André VILLIERS candidat
459 of 1114
Michèle CROUZET candidat
460 of 1114
Ian BOUCARD candidat
461 of 1114
Michel ZUMKELLER candidat
462 of 1114
Manuel VALLS candidat
463 of 1114
Franck MARLIN candidat
464 of 1114
Laëtitia ROMEIRO DIAS candidat
465 of 1114
Marie-Pierre RIXAIN candidat
466 of 1114
Cédric VILLANI candidat
467 of 1114
Amélie DE MONTCHALIN candid

Laure PELLIER candidat
660 of 1114
Paul BERARD candidat
661 of 1114
Latifa CHAY candidat
662 of 1114
Fabienne DELACOUR candidat
663 of 1114
Emmanuel CAMOIN candidat
664 of 1114
Timothée HOUSSIN candidat
665 of 1114
Doris PERREAUX candidat
666 of 1114
Vincent TAILLIEU candidat
667 of 1114
Franck MASSELUS candidat
668 of 1114
Claire Tassadit HOUD candidat
669 of 1114
Harold HUWART candidat
670 of 1114
Clémence ROUVIER candidat
671 of 1114
Jean-Jacques URVOAS candidat
672 of 1114
Pierre-Yves CADALEN candidat
673 of 1114
Jean-Luc BLEUNVEN candidat
674 of 1114
Maël DE CALAN candidat
675 of 1114
Patrick LECLERC candidat
676 of 1114
Gaëlle NICOLAS candidat
677 of 1114
Didier GUILLON candidat
678 of 1114
Michael QUERNEZ candidat
679 of 1114
Yoann GILLET candidat
680 of 1114
. MARIE SARA candidat
681 of 1114
Monique TEZENAS DU MONTCEL candidat
682 of 1114
Brigitte ROULLAUD candidat
683 of 1114
Daniela DE VIDO candidat
684 of 1114
Laurence GARDET candidat
685 of 1114
Claire DUJARDIN candidat
686

Colette CAPDEVIELLE candidat
881 of 1114
Maider AROSTEGUY candidat
882 of 1114
Sylvie FERRER candidat
883 of 1114
Marie-Agnès STARICKY candidat
884 of 1114
Alexandre BOLO candidat
885 of 1114
Christine ESPERT candidat
886 of 1114
Sandrine DOGOR candidat
887 of 1114
Stéphane MASSANELL candidat
888 of 1114
Eric ELKOUBY candidat
889 of 1114
Philippe BIES candidat
890 of 1114
Georges SCHULER candidat
891 of 1114
Sophie ROHFRITSCH candidat
892 of 1114
Gérard SIMLER candidat
893 of 1114
Guy SALOMON candidat
894 of 1114
Antoinette DE SANTIS candidat
895 of 1114
Christian GLIECH candidat
896 of 1114
Etienne WOLF candidat
897 of 1114
Stéphanie VILLEMIN candidat
898 of 1114
Hubert OTT candidat
899 of 1114
Patrick STRIBY candidat
900 of 1114
Aurélie TACQUARD candidat
901 of 1114
Cécile LEHR candidat
902 of 1114
Sylvain MARCELLI candidat
903 of 1114
Elliott AUBIN candidat
904 of 1114
Nathalie PERRIN-GILBERT candidat
905 of 1114
Pascal LE BRUN candidat
906 of 1114
Dominique NACHURY candidat
907 of 

Sylvain BRIAL candidat
1097 of 1114
Moana GREIG candidat
1098 of 1114
Teura IRITI candidat
1099 of 1114
Patrick HOWELL candidat
1100 of 1114
Sonia BACKES candidat
1101 of 1114
Louis MAPOU candidat
1102 of 1114
Frédéric LEFEBVRE candidat
1103 of 1114
Sergio CORONADO candidat
1104 of 1114
Axelle LEMAIRE candidat
1105 of 1114
Sophie RAUSZER candidat
1106 of 1114
François RALLE-ANDREOLI candidat
1107 of 1114
Claudine SCHMID candidat
1108 of 1114
Pierre-Yves LE BORGN' candidat
1109 of 1114
Florence DRORY candidat
1110 of 1114
Leila AÏCHI candidat
1111 of 1114
Alain MARSAUD candidat
1112 of 1114
Thierry MARIANI candidat
1113 of 1114
DONE


In [6]:
all_candidates.to_csv('data/all_candidates_france_2017.csv')